# Small demo notebook for misclassification detection

In this notebook we will train some models on MRPC dataset and explore several popular methods for uncertainty estimation (UE) - MC dropout, Mahalanobis distance and ensembles. So let's start from importing necessary functions and training models.

## Training models

We will train [ELECTRA](https://arxiv.org/abs/2003.10555) model for text classification. But firstly we will import necessary functions. Also don't forget to install dependencies - it's located in ../requirements.txt.

In [1]:
import os
from demo_utils import train_model, preproc_config, train_model_ensemble, get_table, eval_model, calc_mc_dropout
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import open_dict
import hydra
import torch
import numpy as np
from IPython.display import clear_output
import pandas as pd
import json
from pathlib import Path
import logging
from tqdm import tqdm
log = logging.getLogger(__name__)
# if use cuda
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.set_device(0)

2021-12-21 16:07:29.601552: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


To simplify process of setting training parameters, we load predefined config.

In [2]:
# Let's set model params with config
# We will use MRPC dataset
configs_dir = "../configs"
config_path = "mrpc"
abs_config_dir=os.path.abspath(configs_dir)
with initialize_config_dir(config_dir=abs_config_dir):
    config = compose(config_name=config_path)

/home/jovyan/.imgenv-ue-exps-0/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'mrpc': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


Modify config a little before training.

In [3]:
# Set config only to train and train model
config.do_train = True
config.do_eval = True
config.do_ue_estimate = False
config.data.validation_subsample = 0.0
# set dir for saving model and results
with open_dict(config):
    config.model_dir = '../workdir/model/'
# preprocess config - update pathes for saving
config, _, _ = preproc_config(config, init=True)
# seeds
seeds = [42, 4519, 941]
# number of models in ensemble
num_models = 3

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Now we are ready for training models. We will train several models with different seeds, so we could obtain mean values for each experiment.

In [4]:
for seed in seeds:
    config.seed = seed
    train_model(config)
    clear_output()

In [5]:
# also train models for ensemble
train_model_ensemble(config, seeds, num_models)
clear_output()

## Calculating UE

By now, we've trained several models on the MRPC dataset. So now we look at some methods of uncertainty estimation - [MC dropout on all layers](https://proceedings.mlr.press/v48/gal16.html) (MC), [Mahalanobis distance](https://ojs.aaai.org/index.php/AAAI/article/view/17612) (MD) and [Deep ensemble](https://proceedings.neurips.cc/paper/2017/file/9ef2ed4b7fd2c810847ffa5fa85bce38-Paper.pdf).

In [4]:
# Now we have trained model, let's use it and get uncertainty estimation scores
# Set config to UE
config.do_train = False
config.do_eval = True
config.do_ue_estimate = True

At first, create main function for uncertainty estimation. In this function we simply create estimator and call it.

In [5]:
def create_ue_estimator(
    model,
    ue_args,
    eval_metric,
    calibration_dataset,
    train_dataset,
    cache_dir,
    config=None,
):
    if ue_args.ue_type == "mc" or ue_args.ue_type == "mc-dc":
        return UeEstimatorMc(
            model, ue_args, eval_metric, calibration_dataset, train_dataset
        )
    elif ue_args.ue_type == "maha":
        return UeEstimatorMahalanobis(model, ue_args, config, train_dataset)
    else:
        raise ValueError()


def estimate(
    config,
    classifier,
    eval_metric,
    calibration_dataset,
    train_dataset,
    eval_dataset,
    eval_results,
    work_dir
):
    """Function for uncertainty estimation"""
    true_labels = eval_results["true_labels"]
    # create estimator
    ue_estimator = create_ue_estimator(
        classifier,
        config.ue,
        eval_metric,
        calibration_dataset=calibration_dataset,
        train_dataset=train_dataset,
        cache_dir=config.cache_dir,
        config=config,
    )
    # calc UE
    ue_results = ue_estimator(eval_dataset, true_labels)
    # save results
    eval_results.update(ue_results)
    with open(Path(work_dir) / "dev_inference.json", "w") as res:
        json.dump(eval_results, res)

### MC Dropout

Now we implement estimators for each method. Let's start from MC dropout.\
Firstly, we write custom dropout layer and functions for replacing all model dropouts to our custom version.

In [6]:
class DropoutMC(torch.nn.Module):
    def __init__(self, p: float, activate=False):
        super().__init__()
        self.activate = activate
        self.p = p
        self.p_init = p

    def forward(self, x: torch.Tensor):
        return torch.nn.functional.dropout(
            x, self.p, training=self.training or self.activate
        )

In [7]:
def convert_dropouts(model, ue_args):
    """This function replace all model dropouts with custom dropout layer."""
    dropout_ctor = lambda p, activate: DropoutMC(
        p=ue_args.inference_prob, activate=False
    )
    convert_to_mc_dropout(model, {"Dropout": dropout_ctor, "StableDropout": dropout_ctor})

In [8]:

def convert_to_mc_dropout(
    model, substitution_dict
):
    for i, layer in enumerate(list(model.children())):
        proba_field_name = "dropout_rate" if "flair" in str(type(layer)) else "p"
        module_name = list(model._modules.items())[i][0]
        layer_name = layer._get_name()
        proba_field_name = "drop_prob" if layer_name == "StableDropout" else proba_field_name #DeBERTA case
        if layer_name in substitution_dict.keys():
            model._modules[module_name] = substitution_dict[layer_name](
                p=getattr(layer, proba_field_name), activate=False
            )
        else:
            convert_to_mc_dropout(model=layer, substitution_dict=substitution_dict)


def activate_mc_dropout(
    model: torch.nn.Module, activate: bool, random: float = 0.0, verbose: bool = False
):
    for layer in model.children():
        if isinstance(layer, DropoutMC):
            if verbose:
                print(layer)
                print(f"Current DO state: {layer.activate}")
                print(f"Switching state to: {activate}")
            layer.activate = activate
            if activate and random:
                layer.p = random
            if not activate:
                layer.p = layer.p_init
        else:
            activate_mc_dropout(
                model=layer, activate=activate, random=random, verbose=verbose
            )

Now implement estimator for MC Dropout - in this class we simply replace all dropouts with custom layers, activate them, and calc model's predictions several times.

In [9]:
class UeEstimatorMc:
    def __init__(self, cls, ue_args, eval_metric, calibration_dataset, train_dataset):
        self.cls = cls
        self.ue_args = ue_args
        self.calibration_dataset = calibration_dataset
        self.eval_metric = eval_metric
        self.train_dataset = train_dataset

    def __call__(self, eval_dataset, true_labels=None):
        ue_args = self.ue_args
        eval_metric = self.eval_metric
        model = self.cls._auto_model

        log.info("******Perform stochastic inference...*******")

        #if ue_args.dropout_type == "DC_MC":
        #    activate_mc_dropconnect(model, activate=True, random=ue_args.inference_prob)
        #else:
        convert_dropouts(model, ue_args)
        activate_mc_dropout(model, activate=True, random=ue_args.inference_prob)

        if ue_args.use_cache:
            log.info("Caching enabled.")
            model.enable_cache()

        eval_results = {}
        eval_results["sampled_probabilities"] = []
        eval_results["sampled_answers"] = []

        log.info("****************Start runs**************")

        for i in tqdm(range(ue_args.committee_size)):
            preds, probs = self.cls.predict(eval_dataset)[:2]

            eval_results["sampled_probabilities"].append(probs.tolist())
            eval_results["sampled_answers"].append(preds.tolist())

            if ue_args.eval_passes:
                eval_score = eval_metric.compute(
                    predictions=preds, references=true_labels
                )
                log.info(f"Eval score: {eval_score}")

        log.info("**************Done.********************")

        activate_mc_dropout(model, activate=False)

        return eval_results

Now we only have to modify our config and add to it parameters for MC dropout.

In [10]:
config, args_train, args_data = preproc_config(config)
config.ue.ue_type = 'mc'
config.ue.dropout_type = 'MC'
config.ue.inference_prob = 0.1
config.ue.committee_size = 20
config.ue.dropout_subs = 'all'
config.ue.use_cache = True
config.ue.eval_passes = False
config.ue.calibrate = False
config.ue.use_selective = False
for seed in seeds:
    config.seed = seed
    # eval model
    classifier, eval_metric, calibration_dataset, train_dataset, eval_dataset, eval_results, work_dir = eval_model(config, 'mc', args_train, args_data)
    # after calc UE for model
    estimate(config, classifier, eval_metric, calibration_dataset, train_dataset, eval_dataset, eval_results, work_dir)
    clear_output()

### Mahalanobis Distance

We will create estimator for Mahalanobis distance.

Mahalanobis distance is a generalisation of the Euclidean distance, which takes into account the spreading of instances in the training set along various directions in a feature space. $u_{MD} = \min_{c \in C}(h_{i}-\mu_{c})^{T}\Sigma^{-1}(h_{i}-\mu_{c}),$\
where $h_{i}$ is a hidden representation of a $i$-th instance, $\mu_{c}$ is a centroid of a class $c$, and $\Sigma$ is a covariance matrix for hidden representations of training instances.

In [11]:
# This is a liitle modified ELECTRA head, we will use it for extracting model's features.
class ElectraClassificationHeadIdentityPooler(torch.nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, other):
        super().__init__()
        self.dropout1 = other.dropout1
        self.dense = other.dense

    def forward(self, features):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout1(x)
        x = self.dense(x)
        return x

Functions for calculating mahalanobis distance.

In [12]:
def compute_centroids(train_features, train_labels):
    centroids = []
    for label in np.sort(np.unique(train_labels)):
        centroids.append(train_features[train_labels == label].mean(axis=0))
    return np.asarray(centroids)


def compute_covariance(centroids, train_features, train_labels):
    cov = np.zeros((train_features.shape[1], train_features.shape[1]))
    for c, mu_c in tqdm(enumerate(centroids)):
        for x in train_features[train_labels == c]:
            d = (x - mu_c)[:, None]
            cov += d @ d.T
    return cov / train_features.shape[0]


def mahalanobis_distance(train_features, train_labels, eval_features):
    centroids = compute_centroids(train_features, train_labels)
    sigma = compute_covariance(centroids, train_features, train_labels)
    diff = eval_features[:, None, :] - centroids[None, :, :]
    try:
        sigma_inv = np.linalg.inv(sigma)
    except:
        sigma_inv = np.linalg.pinv(sigma)
        log.info("Compute pseudo-inverse matrix")
    dists = np.matmul(np.matmul(diff, sigma_inv), diff.transpose(0, 2, 1))
    dists = np.asarray([np.diag(dist) for dist in dists])
    return np.min(dists, axis=1)

In [13]:
class UeEstimatorMahalanobis:
    def __init__(self, cls, ue_args, config, train_dataset):
        self.cls = cls
        self.ue_args = ue_args
        self.config = config
        self.train_dataset = train_dataset

    def __call__(self, eval_dataset, true_labels=None):
        # because we use ELECTRA
        self.cls.model.classifier = ElectraClassificationHeadIdentityPooler(
            self.cls.model.classifier
        )
        log.info("Change classifier to Identity Pooler")

        eval_labels = [example["label"] for example in eval_dataset]
        eval_dataset = eval_dataset.remove_columns("label")
        eval_features = self.cls.predict(
            eval_dataset, apply_softmax=False, return_preds=False
        )[0]

        train_labels = np.asarray([example["label"] for example in self.train_dataset])
        try:
            self.train_dataset = self.train_dataset.remove_columns("label")
        except:
            self.train_dataset.dataset = self.train_dataset.dataset.remove_columns(
                "label"
            )
        train_features = self.cls.predict(
            self.train_dataset, apply_softmax=False, return_preds=False
        )[0]

        eval_results = {}
        eval_results["eval_labels"] = true_labels
        eval_results["mahalanobis_distance"] = mahalanobis_distance(
            train_features, train_labels, eval_features
        ).tolist()

        log.info("Done.")

        return eval_results

In [14]:
config, args_train, args_data = preproc_config(config)
config.ue.ue_type = 'maha'
config.ue.dropout_type = ''
config.ue.dropout_subs = ''
config.ue.use_cache = True
config.ue.eval_passes = False
config.ue.calibrate = False
config.ue.use_selective = False
for seed in seeds:
    config.seed = seed
    # eval model
    classifier, eval_metric, calibration_dataset, train_dataset, eval_dataset, eval_results, work_dir = eval_model(config, 'maha', args_train, args_data)
    # after calc UE for model
    estimate(config, classifier, eval_metric, calibration_dataset, train_dataset, eval_dataset, eval_results, work_dir)
    clear_output()

### Deep Ensemble

The main idea of ensembles is quite simple - we just run several models and after stack results for each run. So here we don't need to write estimator, we will reuse eval results from training.

In [15]:
def accumulate_results(results_dir, final_dir):
    final_result = {
        "true_labels": [],
        "probabilities": [],
        "answers": [],
        "sampled_probabilities": [],
        "sampled_answers": [],
    }

    for seed in os.listdir(results_dir):
        results_file_path = Path(results_dir) / seed / "dev_inference.json"
        with open(results_file_path) as f:
            result = json.load(f)

        final_result["sampled_probabilities"].append(result["probabilities"])
        final_result["sampled_answers"].append(result["answers"])

    final_result["true_labels"] = result["true_labels"]
    final_result["answers"] = result["answers"]
    final_result["probabilities"] = result["probabilities"]

    with open(Path(final_dir) / "dev_inference.json", "w") as f:
        json.dump(final_result, f)


def calc_ensemble(config, seeds):
    # here we simply stack model predictions
    config, args_train, args_data = preproc_config(config)
    # we assume that we have several trained and evaluated models
    # so we just copy saved models output and stack it
    # get dir with models by seeds
    for idx in os.listdir(os.path.join(config.model_dir, 'ensemble')):
        results_dir = os.path.join(config.model_dir, 'ensemble', idx)
        ensemble_save_dir = os.path.join(config.output_dir, 'ensemble', idx)
        os.makedirs(ensemble_save_dir, exist_ok=True)
        accumulate_results(results_dir, ensemble_save_dir)

In [16]:
# for ensemble we have a slightly different procedure - we already trained models for several seeds
# so now we just reuse results from training
calc_ensemble(config, seeds)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


## Results

We already trained models and calculated uncertainty estimates, it's time to look at results.

In [17]:
def to_table(results, name, return_baseline=False):
    """Small helper function for building tables."""
    replace_dict = {'bald': 'BALD',
                    'variance': 'PV',
                    'sampled_max_prob': 'SMP',
                    'mahalanobis_distance': 'MD',
                    'sampled_entropy': 'SE',
                    'var_ratio': 'VR'}
    table = pd.DataFrame().from_dict(results)
    scores = list(results[list(results.keys())[0]].index)
    scores = [replace_dict.get(score, score) for score in scores]
    table['UE Score'] = scores
    table['Method'] = [name] * len(table)
    table.drop('count', inplace=True)
    baseline = table.loc['baseline (max_prob)']
    baseline['Method'] = 'SR (baseline)'
    baseline['UE Score'] = 'MP'
    table.drop('baseline (max_prob)', inplace=True)
    if return_baseline:
        return table, baseline
    else:
        return table

Here we use several different scores for uncertainty estimation - BALD, probability variance (PV), maximum probability (MP) and sampled maximum probability (SMP).\
The most simple score is MP:\
$u_{SR}(x) = 1 - \max_{c \in C} p(y=c|x).$\
As $C$ in this equation denoted number of classes, and $p(y|x)$ is probability over classes.\
SMP score formulated as following:\
$u_{SMP} = 1 -  \max_{c \in C} \frac{1}{T}\sum_{t=1}^T  p_t^{c}.$\
For this and for all next scores we assume that we have conducted $T$ stochastic passes. In this equation $p_t^{c}$ is the probability of the class $c$ for the $t$-th stochastic forward pass.\
PV score:
$u_{PV} = \frac{1}{C} \sum_{c = 1}^C \left( \frac{1}{T - 1} \sum_{t = 1}^T {(p^c_t-\overline{p^c})^2} \right),$\
where $\overline{p^c}=\frac{1}{T} \sum_t{p^{c}_{t}}$ is the probability for a class $c$ averaged across $T$ stochastic forward passes.\
BALD score:\
$u_{B} = -\sum_{c = 1}^C \overline{p^c} \log \overline{p^c} +  \frac{1}{T}\sum_{c, t} p^{c}_{t}\log p^{c}_{t}.$

In [18]:
# Firstly, let's set pathes to our results
mc_path = os.path.join(config.output_dir, "mc")
maha_path = os.path.join(config.output_dir, "maha")
ensemble_path = os.path.join(config.output_dir, "ensemble")

For evaluating quality of UE we will use reversed pair proportion [(RPP)](https://aclanthology.org/2021.acl-long.84/) and area under the risk-coverage curve [(RCC-AUC)](https://www.jmlr.org/papers/volume11/el-yaniv10a/el-yaniv10a.pdf) metrics.\
The risk coverage curve demonstrates the cumulative sum of loss due to misclassification (cumulative risk) depending on the uncertainty level used for rejection of predictions. The lower area under this curve indicates better quality of the UE method.\
RPP measures how far the uncertainty estimator $\tilde{u}$ is to ideal, given the labeled dataset of size $n$:

$RPP = \frac{1}{n^2} \displaystyle\sum_{i,j = 1}^n \mathbb{1}[\tilde{u}(x_i) > \tilde{u}(x_j), l_i > l_j].$

For both metrics,  $l$ is an indicator loss function.

In [19]:
# We will calculate two metrics - RPP and RCC-AUC
metrics = ['rpp', 'rcc-auc']
mc_res = get_table(config, mc_path)
maha_res = get_table(config, maha_path)
ens_res = get_table(config, ensemble_path)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
PyTorch: set

In [20]:
mc_table, baseline = to_table(mc_res, 'MC', True)
maha_table = to_table(maha_res, 'MD')
ens_table = to_table(ens_res, 'Deep Ensemble')
table = pd.concat([mc_table, maha_table, ens_table])
table.loc['Baseline'] = baseline
# Now make table more readable
columns = ['Method', 'UE Score'] + metrics
table = table[columns]
table.reset_index(drop=True, inplace=True)

In [21]:
table

,Method,UE Score,rpp,rcc-auc
0,MC,BALD,2.27±0.31,18.31±3.67
1,MC,SMP,1.78±0.31,12.70±2.09
2,MC,PV,1.98±0.36,15.18±3.25
3,MD,MD,1.77±0.20,12.48±1.38
4,Deep Ensemble,BALD,1.60±0.11,14.77±0.94
5,Deep Ensemble,SMP,1.47±0.16,11.25±0.33
6,Deep Ensemble,PV,1.50±0.19,13.20±2.33
7,SR (baseline),MP,2.00±0.46,15.87±4.93


As one can see, Deep Ensemble shows the best performance, but requires a lot of computational time. The second best method is Mahalanobis distance (MD), that shows good results and works faster.